#### 문제 2-3 : 학생 정보 구조화 시스템

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_w


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 출력 구조를 정의하는 Pydantic 모델
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이 (정수)")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 리스트")
    goal: str = Field(description="학생의 목표")

In [ ]:
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentInfo)

In [11]:
# 프롬프트 템플릿 설정
template = """
당신은 학생의 자기소개 텍스트를 분석하는 전문가입니다.
다음 자유 형식의 자기소개에서 학생 정보를 추출하여 JSON 형식으로 구조화하세요.

자기소개: {intro}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

In [ ]:
# 환경변수에서 OpenAI API 키 로드 (실제 사용시 주석 해제)
# import os
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

In [8]:
# 체인 구성 및 실행
intro = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

chain = prompt | model | parser
output = chain.invoke({"intro": intro})

print(type(output))
print(output)

<class '__main__.StudentInfo'>
name='김민수' age=22 major='컴퓨터공학' hobbies=['게임하기', '영화보기', '코딩'] goal='훌륭한 개발자가 되는 것'


In [9]:
# 결과 출력
print(f"이름: {output.name}")
print(f"나이: {output.age}")
print(f"전공: {output.major}")
print(f"취미: {', '.join(output.hobbies)}")
print(f"목표: {output.goal}")

이름: 김민수
나이: 22
전공: 컴퓨터공학
취미: 게임하기, 영화보기, 코딩
목표: 훌륭한 개발자가 되는 것
